- 데이터셋:
    * 총 운항 기록 24/07/01~ 24/12/31
    * 총 이상운항 기록 24/07/01 ~ 24/12/31
- 가설 : 각종 원인으로 인한 지연 발생 후 접속원인으로 발생하는 지연사고가 자주 발생한다.

검증 목표 : 총 운항기록중 시간대별로 여객기와 화물수송기를 분류, 최초지연과 그 이후 이어지는 지연사건이 몇건이나 되는지 확인하여, 지연사고 최초발생과 그 이후 생기는 연결/접속 지연사고의 관계성을 밝힘



In [2]:
## [1] 모듈 로딩
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st

import utils

In [3]:
## 데이터 로딩및 확인
dataDF07 = pd.read_excel('./DATA/normal/항공기출도착현황240701.xlsx')

# 계획시간 별로 소트하여 지연 현황을 찾고, 해당 지연 원인을 abnormal 폴더에서 해당날짜와 같은 날짜에서 찾아오기
dataDF07.head()

c:\Users\kdt\anaconda3\envs\PY39\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,출/도착구분,날짜,항공사,편명,출발공항코드,출발공항명,도착공항코드,도착공항명,계획시간,예상시간,도착시간,구분,현황
0,도착,20240701,에어아시아버하드,AK1623,BKI,코타키나발루,ICN,인천,00:05,00:31,00:25,여객,지연
1,도착,20240701,아틀라스화물항공,5Y555,JFK,뉴욕,ICN,인천,00:05,00:20,00:12,화물,지연
2,도착,20240701,카고룩스 이탈리아,C87736,MXP,밀라노,ICN,인천,00:05,00:22,00:16,화물,지연
3,도착,20240701,폴라에어카고,PO213,CVG,신시내티,ICN,인천,00:35,21:35,:,화물,취소
4,도착,20240701,몽골항공,OM309,UBN,칭기즈 칸(신 울란바토르) 국제공항,ICN,인천,00:40,00:40,00:32,여객,도착


In [4]:
dataDF07['현황'].value_counts()

현황
도착    10591
지연     7621
취소      245
회항       41
Name: count, dtype: int64

In [5]:
dataDF07.shape

(18499, 13)

In [6]:
abDF0701 = pd.read_csv('./DATA/abnormal/filtered_data_20240701.csv')
abDF0701['비정상원인'].unique()

array(['항공기 연결(도착지연)', '탑승(승객 불일치, 승객 수속오류, 수속 완료된 승객 소재불명)', nan,
       '기타(지연코드 외)', '수속오류(승객 및 수화물)', '기타(접속 관련)', '항공기 결함',
       '정시 출발하였으나 해당 공항에 접속 지연', '공항, 관제 사유 접속 지연',
       '운항통제(항로변경, 회항, 항공기 교체)', '항공기 정비(정비 스케줄 지연)',
       'ATFM(항공로 공표 수용능력 저하)', '항공기 결함(DF, DG) 사유 접속 지연', '기상 사유 접속 지연',
       '화물 연결', 'ULD(컨테이너 등)', '[기상]기타 기상 악화', '출발공항 제한사항(관제, 활주로폐쇄 등)',
       '기타(화물 관련)', '기타(지상조업 관련)', '항공기 정비(계획되지 않은 정비)', '정비 사유 접속 지연',
       '위탁수화물처리(분류 등)', '수화물 조업지연', '기타(여객 및 위탁수화물 관련)',
       '공항시설(주기장 부족, 혼잡 등)'], dtype=object)

In [7]:
abDF0701[100:150]

,항공사,항공편명,목적지,계획,예상,출발,구분,현황,비정상원인,비고
100,에어서울,RS706,NRT(도쿄/나리타),14:10,14:36,14:28,여객,지연,NaN,NaN
101,대한항공,KE894,PVG(푸동),14:20,14:27,14:15,여객,지연,NaN,NaN
102,대한항공,KE260,ORD(시카고),14:25,18:19,18:07,화물,지연,기상 사유 접속 지연,NaN
103,칼리타항공,K4527,HKG(홍콩),14:25,18:19,18:09,NaN,지연,항공기 연결(도착지연),NaN
104,내셔널 항공,N8664,CAN(광저우),14:25,:,:,NaN,취소,화물 연결,NaN
105,델타항공,DL189,ATL(애틀랜타),14:25,15:04,14:59,여객,지연,항공기 정비(계획되지 않은 정비),NaN
106,아틀라스화물항공,5Y8545,JFK(뉴욕),14:25,21:25,:,화물,취소,기타(지연코드 외),NaN
107,아시아나항공,OZ362,PVG(푸동),14:30,14:57,14:45,여객,지연,NaN,NaN
108,아시아나항공,OZ562,FCO(로마),14:40,14:56,14:41,여객,지연,NaN,NaN
109,제주항공,7C9304,YNT(연대),14:55,15:28,15:18,화물,지연,"공항, 관제 사유 접속 지연",NaN


In [8]:
# lateDate = range(1001,1032)
for lateDate in range(20241001,20241032):
    lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
    print(lateDate, (lateDay['비정상원인'].str.contains('기상 사유')).count())
    

20241001 95
20241002 107
20241003 104
20241004 139
20241005 75
20241006 92
20241007 92
20241008 97
20241009 105
20241010 96
20241011 81
20241012 58
20241013 77
20241014 107
20241015 81
20241016 90
20241017 85
20241018 133
20241019 86
20241020 122
20241021 113
20241022 123
20241023 135
20241024 111
20241025 117
20241026 121
20241027 106
20241028 106
20241029 92
20241030 91
20241031 108


In [12]:
tlDF = pd.read_csv('./DATA/result/total_late.csv', index_col=0)

In [14]:
tlDF

,기상,연결,정비,여객 및 화물 처리,지상조업,운항기준/승무원,항공교통흐름,공항/출입국절차,기타,총 지연편수
24년,,,,,,,,,,
1월,206,"3,748",245,384,89,51,"1,246","1,316",288,"7,573"
2월,302,"3,877",217,341,102,58,968,"1,048",306,"7,219"
3월,130,"2,894",213,280,83,51,890,953,344,"5,838"
4월,139,"2,780",195,268,59,55,800,"1,058",323,"5,677"
5월,159,"3,095",198,227,69,49,"1,164","1,370",277,"6,608"
6월,152,"4,664",209,230,73,52,"1,949","2,248",444,"10,021"
7월,457,"7,025",245,296,114,60,"2,643","2,758",538,"14,136"
8월,382,"5,647",262,274,115,131,"2,306","2,894",439,"12,450"
9월,179,"4,336",270,262,86,95,"2,017","2,509",484,"10,238"


In [20]:
tlDF['기상'].corr(tlDF['총 지연편수'])
# pd.corr(tlDF['기상'], tlDF['총 지연편수'])

ValueError: could not convert string to float: '2,928'